In [6]:
####USing the Convolutional Block Attention Module (CBAM). 
##CBAM is a lightweight and effective attention mechanism that can be easily integrated into existing architectures. 
!pip install ultralytics --upgrade


In [2]:
###Training with Attention Module
#To enhance the YOLO11n model with an additional attention module for instance segmentation, 
#we can incorporate the Convolutional Block Attention Module (CBAM). 
#CBAM is an effective attention mechanism that can improve the model's 
#performance by focusing on important features in both spatial and channel dimensions.
import torch
import torch.nn as nn
from ultralytics import YOLO

# Define the CBAM module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, channel):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(channel)
        self.spatial_attention = SpatialAttention()

    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

# Modify YOLO11n model to include CBAM
class YOLO11nCBAM(YOLO):
    def __init__(self, model_path):
        super().__init__(model_path)
        
        # Add CBAM modules to the model
        for m in self.model.modules():
            if isinstance(m, nn.Conv2d):
                cbam = CBAM(m.out_channels)
                new_module = nn.Sequential(m, cbam)
                for name, module in self.model.named_modules():
                    if module is m:
                        setattr(self.model, name, new_module)
                        break

# Define paths
dataset_path = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml'
weights_path = 'yolo11n-seg.pt'
output_dir = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment'

# Load the modified YOLO11n model with CBAM
model = YOLO11nCBAM(weights_path)

# Set model training parameters
train_results = model.train(
    data=dataset_path,
    epochs=500,
    imgsz=640,
    batch=8,
    project=output_dir,
    name='train_cbam',
    exist_ok=True
)

print(f"Training outputs and model weights will be saved in: {output_dir}")


##This code introduces the following changes:
#We define the CBAM module, which consists of Channel Attention and Spatial Attention components
#We create a custom YOLO11nCBAM class that inherits from the YOLO class and adds CBAM modules after each convolutional layer in the model
#We instantiate the modified model using YOLO11nCBAM instead of the standard YOLO class.

##The CBAM attention module will help the model focus on important features in both channel and spatial dimensions, potentially improving the accuracy of instance segmentation tasks
##Keep in mind that adding attention modules may increase the computational complexity and training time of the model. You might need to adjust the batch size or use a more powerful GPU if you encounter memory issues.
##Also, note that this modification affects the entire model architecture. Depending on your specific requirements, you might want to add CBAM modules only to certain layers or parts of the network for optimal performance

/home/mzc/anaconda3/lib/python3.11/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Ultralytics 8.3.28 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml, epochs=500, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment, name=train_cbam, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False

train: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11A

train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/output12_3_00460_jpg.rf.6e57ea340cdf28d34aafc40f7a60652c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree10_00300_jpg.rf.1d4a4a5eebb14913bb0759c4a140a27f.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree12_00700_jpg.rf.49df6e487eb27e6617ecbab3ae62d674.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree14_00500_jpg.rf.ad4afdad3cf7729abf490dc8ca495195.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.


val: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11Att

...

                   all         86       1989      0.751      0.628      0.661      0.401      0.727      0.565      0.615      0.307

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    485/500      2.49G     0.9822       1.23     0.6216     0.9016        306   
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         86       1989      0.743       0.63      0.659        0.4      0.712      0.567      0.613      0.305
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 385, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

485 epochs completed in 2.140 hours.
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/train_cbam/weights/last.pt, 6.1MB
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/train_cbam/weights/best.pt, 6.1MB

Validating /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/train_cbam/weights/best.pt...
Ultralytics 8.3.28 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
YOLO11n-seg s

                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         86       1989      0.742      0.622      0.654      0.402      0.717      0.576      0.623      0.315
                Branch         86       1268      0.693      0.533      0.563      0.295      0.643      0.468      0.504      0.209
                 Trunk         86        721      0.791       0.71      0.744      0.508      0.792      0.685      0.742      0.421
Speed: 0.3ms preprocess, 3.0ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/train_cbam
Training outputs and model weights will be saved in: /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment


In [24]:
###Training with Attention Module
#To enhance the YOLO11n model with an additional attention module for instance segmentation, 
#we can incorporate the Convolutional Block Attention Module (CBAM). 
#CBAM is an effective attention mechanism that can improve the model's 
#performance by focusing on important features in both spatial and channel dimensions.
import torch
import torch.nn as nn
from ultralytics import YOLO

# Define the CBAM module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, channel):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(channel)
        self.spatial_attention = SpatialAttention()

    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

# Modify YOLO11n model to include CBAM
class YOLO11nCBAM(YOLO):
    def __init__(self, model_path):
        super().__init__(model_path)
        
        # Add CBAM modules to the model
        for m in self.model.modules():
            if isinstance(m, nn.Conv2d):
                cbam = CBAM(m.out_channels)
                new_module = nn.Sequential(m, cbam)
                for name, module in self.model.named_modules():
                    if module is m:
                        setattr(self.model, name, new_module)
                        break

# Define paths
dataset_path = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml'
weights_path = 'yolo11s-seg.pt'
output_dir = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/S'

# Load the modified YOLO11n model with CBAM
model = YOLO11nCBAM(weights_path)

# Set model training parameters
train_results = model.train(
    data=dataset_path,
    epochs=500,
    imgsz=640,
    batch=8,
    project=output_dir,
    name='train_cbam',
    exist_ok=True
)

print(f"Training outputs and model weights will be saved in: {output_dir}")


##This code introduces the following changes:
#We define the CBAM module, which consists of Channel Attention and Spatial Attention components
#We create a custom YOLO11nCBAM class that inherits from the YOLO class and adds CBAM modules after each convolutional layer in the model
#We instantiate the modified model using YOLO11nCBAM instead of the standard YOLO class.

##The CBAM attention module will help the model focus on important features in both channel and spatial dimensions, potentially improving the accuracy of instance segmentation tasks
##Keep in mind that adding attention modules may increase the computational complexity and training time of the model. You might need to adjust the batch size or use a more powerful GPU if you encounter memory issues.
##Also, note that this modification affects the entire model architecture. Depending on your specific requirements, you might want to add CBAM modules only to certain layers or parts of the network for optimal performance

100%|██████████████████████████████████████| 19.7M/19.7M [00:00<00:00, 41.0MB/s]
/home/mzc/anaconda3/lib/python3.11/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Ultralytics 8.3.29 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml, epochs=500, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/S, name=train_cbam, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fal

train: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11A

train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/output12_3_00460_jpg.rf.6e57ea340cdf28d34aafc40f7a60652c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree10_00300_jpg.rf.1d4a4a5eebb14913bb0759c4a140a27f.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree12_00700_jpg.rf.49df6e487eb27e6617ecbab3ae62d674.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree14_00500_jpg.rf.ad4afdad3cf7729abf490dc8ca495195.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.


val: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11Att

...

    427/500      3.42G     0.8038      1.049     0.5013      0.857        303   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         86       1989      0.721       0.66      0.677      0.426      0.712       0.57      0.611      0.319
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 327, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



427 epochs completed in 3.037 hours.
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/S/train_cbam/weights/last.pt, 20.6MB
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/S/train_cbam/weights/best.pt, 20.6MB

Validating /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/S/train_cbam/weights/best.pt...
Ultralytics 8.3.29 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
YOLO11s-seg summary (fused): 265 layers, 10,067,590 parameters, 0 gradients, 35.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         86       1989      0.762      0.632       0.68      0.426      0.717      0.589       0.63      0.329
                Branch         86       1268      0.715      0.547      0.589      0.315      0.637      0.479      0.508      0.217
                 Trunk         86        721      0.809      0.717      0.771      0.536      0.796      0.699      0.753      0.442
Speed: 0.2ms preprocess, 5.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/S/train_cbam
Training outputs and model weights will be saved in: /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/S


In [25]:
###Training with Attention Module
#To enhance the YOLO11n model with an additional attention module for instance segmentation, 
#we can incorporate the Convolutional Block Attention Module (CBAM). 
#CBAM is an effective attention mechanism that can improve the model's 
#performance by focusing on important features in both spatial and channel dimensions.
import torch
import torch.nn as nn
from ultralytics import YOLO

# Define the CBAM module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, channel):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(channel)
        self.spatial_attention = SpatialAttention()

    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

# Modify YOLO11n model to include CBAM
class YOLO11nCBAM(YOLO):
    def __init__(self, model_path):
        super().__init__(model_path)
        
        # Add CBAM modules to the model
        for m in self.model.modules():
            if isinstance(m, nn.Conv2d):
                cbam = CBAM(m.out_channels)
                new_module = nn.Sequential(m, cbam)
                for name, module in self.model.named_modules():
                    if module is m:
                        setattr(self.model, name, new_module)
                        break

# Define paths
dataset_path = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml'
weights_path = 'yolo11m-seg.pt'
output_dir = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/M'

# Load the modified YOLO11n model with CBAM
model = YOLO11nCBAM(weights_path)

# Set model training parameters
train_results = model.train(
    data=dataset_path,
    epochs=500,
    imgsz=640,
    batch=8,
    project=output_dir,
    name='train_cbam',
    exist_ok=True
)

print(f"Training outputs and model weights will be saved in: {output_dir}")


##This code introduces the following changes:
#We define the CBAM module, which consists of Channel Attention and Spatial Attention components
#We create a custom YOLO11nCBAM class that inherits from the YOLO class and adds CBAM modules after each convolutional layer in the model
#We instantiate the modified model using YOLO11nCBAM instead of the standard YOLO class.

##The CBAM attention module will help the model focus on important features in both channel and spatial dimensions, potentially improving the accuracy of instance segmentation tasks
##Keep in mind that adding attention modules may increase the computational complexity and training time of the model. You might need to adjust the batch size or use a more powerful GPU if you encounter memory issues.
##Also, note that this modification affects the entire model architecture. Depending on your specific requirements, you might want to add CBAM modules only to certain layers or parts of the network for optimal performance

100%|██████████████████████████████████████| 43.3M/43.3M [00:01<00:00, 31.6MB/s]
/home/mzc/anaconda3/lib/python3.11/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Ultralytics 8.3.29 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
engine/trainer: task=segment, mode=train, model=yolo11m-seg.pt, data=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml, epochs=500, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/M, name=train_cbam, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fal

train: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11A

train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/output12_3_00460_jpg.rf.6e57ea340cdf28d34aafc40f7a60652c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree10_00300_jpg.rf.1d4a4a5eebb14913bb0759c4a140a27f.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree12_00700_jpg.rf.49df6e487eb27e6617ecbab3ae62d674.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree14_00500_jpg.rf.ad4afdad3cf7729abf490dc8ca495195.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.


val: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11Att

...

    500/500      6.74G     0.6044     0.8574     0.3667     0.8144        149   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         86       1989      0.772       0.65      0.684      0.447      0.713      0.589      0.623      0.337



500 epochs completed in 7.373 hours.
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/M/train_cbam/weights/last.pt, 45.2MB
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/M/train_cbam/weights/best.pt, 45.2MB

Validating /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/M/train_cbam/weights/best.pt...
Ultralytics 8.3.29 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
YOLO11m-seg summary (fused): 330 layers, 22,336,854 parameters, 0 gradients, 123.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         86       1989      0.766      0.661      0.698      0.455      0.749      0.581      0.636      0.347
                Branch         86       1268      0.707      0.584      0.613      0.345      0.669      0.475      0.516      0.232
                 Trunk         86        721      0.825      0.738      0.783      0.565       0.83      0.688      0.755      0.462
Speed: 0.2ms preprocess, 11.7ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/M/train_cbam
Training outputs and model weights will be saved in: /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/M


In [1]:
###Training with Attention Module
#To enhance the YOLO11n model with an additional attention module for instance segmentation, 
#we can incorporate the Convolutional Block Attention Module (CBAM). 
#CBAM is an effective attention mechanism that can improve the model's 
#performance by focusing on important features in both spatial and channel dimensions.
import torch
import torch.nn as nn
from ultralytics import YOLO

# Define the CBAM module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, channel):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(channel)
        self.spatial_attention = SpatialAttention()

    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

# Modify YOLO11n model to include CBAM
class YOLO11nCBAM(YOLO):
    def __init__(self, model_path):
        super().__init__(model_path)
        
        # Add CBAM modules to the model
        for m in self.model.modules():
            if isinstance(m, nn.Conv2d):
                cbam = CBAM(m.out_channels)
                new_module = nn.Sequential(m, cbam)
                for name, module in self.model.named_modules():
                    if module is m:
                        setattr(self.model, name, new_module)
                        break

# Define paths
dataset_path = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml'
weights_path = 'yolo11l-seg.pt'
output_dir = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/L'

# Load the modified YOLO11n model with CBAM
model = YOLO11nCBAM(weights_path)

# Set model training parameters
train_results = model.train(
    data=dataset_path,
    epochs=500,
    imgsz=640,
    batch=8,
    project=output_dir,
    name='train_cbam',
    exist_ok=True
)

print(f"Training outputs and model weights will be saved in: {output_dir}")


##This code introduces the following changes:
#We define the CBAM module, which consists of Channel Attention and Spatial Attention components
#We create a custom YOLO11nCBAM class that inherits from the YOLO class and adds CBAM modules after each convolutional layer in the model
#We instantiate the modified model using YOLO11nCBAM instead of the standard YOLO class.

##The CBAM attention module will help the model focus on important features in both channel and spatial dimensions, potentially improving the accuracy of instance segmentation tasks
##Keep in mind that adding attention modules may increase the computational complexity and training time of the model. You might need to adjust the batch size or use a more powerful GPU if you encounter memory issues.
##Also, note that this modification affects the entire model architecture. Depending on your specific requirements, you might want to add CBAM modules only to certain layers or parts of the network for optimal performance

100%|██████████████████████████████████████| 53.5M/53.5M [00:01<00:00, 39.0MB/s]
/home/mzc/anaconda3/lib/python3.11/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Ultralytics 8.3.29 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
engine/trainer: task=segment, mode=train, model=yolo11l-seg.pt, data=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml, epochs=500, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/L, name=train_cbam, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=Fal

train: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11A

train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/output12_3_00460_jpg.rf.6e57ea340cdf28d34aafc40f7a60652c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree10_00300_jpg.rf.1d4a4a5eebb14913bb0759c4a140a27f.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree12_00700_jpg.rf.49df6e487eb27e6617ecbab3ae62d674.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree14_00500_jpg.rf.ad4afdad3cf7729abf490dc8ca495195.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.


val: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11Att

...

    417/500      7.78G     0.6448     0.8853     0.3867     0.8335        336   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         86       1989      0.767      0.651      0.684      0.449      0.731      0.597      0.634      0.345
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 317, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



417 epochs completed in 7.239 hours.
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/L/train_cbam/weights/last.pt, 55.9MB
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/L/train_cbam/weights/best.pt, 55.9MB

Validating /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/L/train_cbam/weights/best.pt...
Ultralytics 8.3.29 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
YOLO11l-seg summary (fused): 491 layers, 27,586,134 parameters, 0 gradients, 141.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         86       1989       0.77      0.658      0.691      0.449      0.736      0.604      0.644      0.353
                Branch         86       1268       0.72      0.576        0.6      0.334      0.653      0.496      0.514       0.23
                 Trunk         86        721       0.82      0.739      0.782      0.564      0.818      0.713      0.773      0.477
Speed: 0.3ms preprocess, 14.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/L/train_cbam
Training outputs and model weights will be saved in: /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/L


In [2]:
import torch
import torch.nn as nn
from ultralytics import YOLO

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the CBAM module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

class CBAM(nn.Module):
    def __init__(self, channel):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(channel)
        self.spatial_attention = SpatialAttention()

    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

# Modify YOLO11n model to include CBAM
class YOLO11nCBAM(YOLO):
    def __init__(self, model_path):
        super().__init__(model_path)
        
        # Add CBAM modules to the model
        for m in self.model.modules():
            if isinstance(m, nn.Conv2d):
                cbam = CBAM(m.out_channels)
                new_module = nn.Sequential(m, cbam)
                for name, module in self.model.named_modules():
                    if module is m:
                        setattr(self.model, name, new_module)
                        break
        
        # Move the model to GPU
        self.model.to(device)

# Define paths
dataset_path = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml'
weights_path = 'yolo11x-seg.pt'
output_dir = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/X'

# Load the modified YOLO11n model with CBAM
model = YOLO11nCBAM(weights_path)

# Set model training parameters
train_results = model.train(
    data=dataset_path,
    epochs=500,
    imgsz=640,
    batch=8,
    project=output_dir,
    name='train_cbam',
    exist_ok=True,
    device=0 if torch.cuda.is_available() else 'cpu'  # Use GPU if available
)

print(f"Training outputs and model weights will be saved in: {output_dir}")

Using device: cuda


100%|████████████████████████████████████████| 119M/119M [00:04<00:00, 25.9MB/s]
/home/mzc/anaconda3/lib/python3.11/site-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Ultralytics 8.3.29 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
engine/trainer: task=segment, mode=train, model=yolo11x-seg.pt, data=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml, epochs=500, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/X, name=train_cbam, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False,

train: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11A

train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/output12_3_00460_jpg.rf.6e57ea340cdf28d34aafc40f7a60652c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree10_00300_jpg.rf.1d4a4a5eebb14913bb0759c4a140a27f.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree12_00700_jpg.rf.49df6e487eb27e6617ecbab3ae62d674.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree14_00500_jpg.rf.ad4afdad3cf7729abf490dc8ca495195.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.

...

    500/500       9.6G     0.5513     0.8069     0.3261     0.8121        149   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         86       1989      0.769      0.673      0.688      0.455      0.715      0.617      0.636      0.351



500 epochs completed in 15.729 hours.
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/X/train_cbam/weights/last.pt, 124.8MB
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/X/train_cbam/weights/best.pt, 124.8MB

Validating /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/X/train_cbam/weights/best.pt...
Ultralytics 8.3.29 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
YOLO11x-seg summary (fused): 491 layers, 62,004,438 parameters, 0 gradients, 318.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         86       1989      0.762      0.669        0.7      0.459        0.7      0.614      0.641      0.355
                Branch         86       1268      0.733       0.59      0.623      0.344      0.633      0.503      0.514      0.232
                 Trunk         86        721       0.79      0.747      0.776      0.574      0.768      0.724      0.769      0.478
Speed: 0.4ms preprocess, 24.8ms inference, 0.1ms loss, 1.1ms postprocess per image
Results saved to /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/X/train_cbam
Training outputs and model weights will be saved in: /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/X


In [7]:
!pip install ultralytics --upgrade
import os
# Import necessary libraries
from ultralytics import YOLO

# Define paths
dataset_path = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml'
weights_path = 'yolo11n-seg.pt'  
output_dir = '/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/NnoCBAM'

# Load the YOLO model configuration for instance segmentation
model = YOLO(weights_path)

# Set model training parameters
train_results = model.train(
    data=dataset_path,
    epochs=500,  # Number of training epochs
    imgsz=640,  # Image size (resize images to this size during training)
    batch=8,  # Batch size
    project=output_dir,  # Directory to save training outputs and model weights
    name='train',  # Name for the training run (subdirectory within project)
    exist_ok=True  # Overwrite existing project/name if it exists
)

# Print the path to the trained model weights and outputs for your reference
print(f"Training outputs and model weights will be saved in: {output_dir}")


New https://pypi.org/project/ultralytics/8.3.34 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.32 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/data.yaml, epochs=500, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/NnoCBAM, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det

train: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11A

train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/output12_3_00460_jpg.rf.6e57ea340cdf28d34aafc40f7a60652c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree10_00300_jpg.rf.1d4a4a5eebb14913bb0759c4a140a27f.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree12_00700_jpg.rf.49df6e487eb27e6617ecbab3ae62d674.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/train/images/tree14_00500_jpg.rf.ad4afdad3cf7729abf490dc8ca495195.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.


val: Scanning /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11Att

val: WARNING ⚠️ /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/data/TREES.v1i.yolov11 (1)/valid/images/tree6_00800_jpg.rf.85d1e0b48b37eb380e7c2dfe6d1746f3.jpg: 1 duplicate labels removed


...

                   all         86       1989      0.751      0.628      0.661      0.401      0.727      0.565      0.615      0.307

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    485/500      2.57G     0.9822       1.23     0.6216     0.9016        306   
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         86       1989      0.743       0.63      0.659        0.4      0.712      0.567      0.613      0.305
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 385, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

485 epochs completed in 2.462 hours.
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/NnoCBAM/train/weights/last.pt, 6.1MB
Optimizer stripped from /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/NnoCBAM/train/weights/best.pt, 6.1MB

Validating /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/NnoCBAM/train/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.11.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA TITAN Xp COLLECTORS EDITION, 12189MiB)
YOLO

                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         86       1989      0.742      0.622      0.654      0.402      0.717      0.576      0.623      0.315
                Branch         86       1268      0.693      0.533      0.563      0.295      0.643      0.468      0.504      0.209
                 Trunk         86        721      0.791       0.71      0.744      0.508      0.792      0.685      0.742      0.421
Speed: 0.4ms preprocess, 2.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/NnoCBAM/train
Training outputs and model weights will be saved in: /media/mzc/e4fcae9b-7400-4641-a800-034f249f8d14/mzc/TreesYOLO11AttentionMODULE/Experiment/NnoCBAM
